In [32]:
import socket
import json
host = "localhost"
port = 8080

client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect((host,port))
login_info = {'name': "riccardo", "action": "createRoom", "actionSpecs": {"roomName": "A"}}
login_info = json.dumps(login_info)
client.send(bytes(login_info, encoding="ascii"))

73

In [31]:
response = client.recv(4096)
print(response)

b''
